In [ ]:
# This block allows us to import from the benchmark folder,
# as if it was a package installed using pip
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from util import datasets, methods, models
from benchmark.noise_invariance import noise_invariance, NoisePerturbedDataset, generate_noise_perturbed_dataset
from os import path
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
generate_dataset = False
dataset_name = "MNIST_noise"
original_dataset_name = "MNIST"
data_root = "../data"
batch_size = 4
n_batches = 64
use_logits = False
normalize_attrs = True

In [4]:
if original_dataset_name == "MNIST":
    model = models.MNISTCNN(params_loc=path.join(data_root, "models/MNIST_cnn.pth"), output_logits=use_logits)
    model.eval()
    if generate_dataset:
        dataset = datasets.MNIST(batch_size=batch_size, data_location=path.join(data_root, "MNIST"),
                            download=False, shuffle=False)

In [5]:
if original_dataset_name == "CIFAR10":
    model = models.CifarResnet(version="resnet32", 
                               params_loc=path.join(data_root, "models/cifar10_resnet32.pth"),
                               num_classes=10, output_logits=use_logits)
    model.eval()
    if generate_dataset:
        dataset = datasets.Cifar(batch_size=batch_size, data_location=path.join(data_root, "CIFAR10"),
                            download=False, shuffle=False, version="cifar10")

In [6]:
if generate_dataset:
    perturbed_dataset = generate_noise_perturbed_dataset(dataset.get_test_data(),
                                                         path.join(data_root, dataset_name),
                                                         perturbation_levels=list(np.linspace(0, 0.1, 10)),
                                                         n_batches=n_batches,
                                                         model=lambda x: model(x).detach().numpy(), max_tries=2)
else:
    perturbed_dataset = NoisePerturbedDataset(path.join(data_root, dataset_name))

In [7]:
attribution_methods = {
    #"GuidedGradCAM": methods.GuidedGradCAM(model, model.get_last_conv_layer(), normalize=normalize_attrs),
    "Gradient": methods.Gradient(model, normalize=normalize_attrs),
    "InputXGradient": methods.InputXGradient(model, normalize=normalize_attrs),
    "IntegratedGradients": methods.IntegratedGradients(model, normalize=normalize_attrs),
    "GuidedBackprop": methods.GuidedBackprop(model, normalize=normalize_attrs),
    "Deconvolution": methods.Deconvolution(model, normalize=normalize_attrs)
}

In [8]:
result = noise_invariance(perturbed_dataset, attribution_methods)
result["Gradient"]["diffs"].shape

Method: Gradient...


/home/arne/Documents/PhD/projects/benchmark/venv/lib/python3.6/site-packages/captum/attr/_utils/gradient.py:33: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  "required_grads has been set automatically." % index


Method: InputXGradient...


RuntimeError: Can't call numpy() on Variable that requires grad. Use var.detach().numpy() instead.

In [ ]:
result_df = pd.DataFrame.from_dict(
    {m_name: pd.DataFrame(data=result[m_name]["diffs"]).stack() for m_name in attribution_methods}
).stack().reset_index()
result_df.columns = ["sample", "noise", "method", "difference"]
result_df["noise"] = np.array(perturbed_dataset.perturbation_levels)[result_df["noise"]]
result_df

In [ ]:
plt.figure(figsize=(10, 5))
sns.lineplot(x="noise", y="difference", hue="method", data=result_df)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)